Name: VEN THON
>
ID: e20191250
>
Group: I5-GIC-C



In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from prettytable import PrettyTable

In [2]:
with open('Text1.txt', 'r') as f:
  text = f.read().replace('\n', '')

print(text)

It seems like your request is a bit unclear. Could you please provide more details or specify what you're looking for?


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize

sentences = sent_tokenize(text)
print(len(sentences))

2


In [5]:
train_size = int(len(sentences)*0.7)
valid_size = int(len(sentences)*0.1)

train = sentences[:train_size]
valid = sentences[train_size:train_size+valid_size]
test = sentences[train_size+valid_size:]

print(len(train), len(valid), len(test))

1 0 1


In [6]:
tokens = word_tokenize(' '.join(train))
print(tokens)
print(len(tokens))

['It', 'seems', 'like', 'your', 'request', 'is', 'a', 'bit', 'unclear', '.']
10


In [7]:
from collections import Counter

word_freq = Counter(tokens).most_common(50)

print(len(word_freq))

vocab = [w for w, _ in word_freq]
print(vocab)

10
['It', 'seems', 'like', 'your', 'request', 'is', 'a', 'bit', 'unclear', '.']


In [8]:
new_tokens = [_ if _ in vocab else '<unk>' for _ in tokens]
print(new_tokens)

['It', 'seems', 'like', 'your', 'request', 'is', 'a', 'bit', 'unclear', '.']


In [9]:
# Tokenize and remove stopwords
tokens = [word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in stopwords.words('english')]
# Create a frequency distribution
freq_dist = FreqDist(tokens)
# Select top 50 frequent words
top_50_words = [word for word, _ in freq_dist.most_common(50)]

In [10]:
# Create a term-term matrix
context_window = 4
term_term_matrix = {}
for i in range(len(tokens)):
    for j in range(max(0, i - context_window), min(len(tokens), i + context_window + 1)):
        if i != j and tokens[i] in top_50_words and tokens[j] in top_50_words:
            pair = (tokens[i], tokens[j])
            term_term_matrix[pair] = term_term_matrix.get(pair, 0) + 1


In [11]:
# Convert the term-term matrix dictionary to a DataFrame
term_term_matrix_df = pd.DataFrame(index=top_50_words, columns=top_50_words)

for pair, count in term_term_matrix.items():
    term_term_matrix_df.at[pair[0], pair[1]] = count
# Fill NaN values with 0
term_term_matrix_df = term_term_matrix_df.fillna(0)

# Display the term-term matrix DataFrame
print(term_term_matrix_df)



         seems  like  request  bit  unclear  could  please  provide  details  \
seems        0     1        1    1        1      0       0        0        0   
like         1     0        1    1        1      1       0        0        0   
request      1     1        0    1        1      1       1        0        0   
bit          1     1        1    0        1      1       1        1        0   
unclear      1     1        1    1        0      1       1        1        1   
could        0     1        1    1        1      0       1        1        1   
please       0     0        1    1        1      1       0        1        1   
provide      0     0        0    1        1      1       1        0        1   
details      0     0        0    0        1      1       1        1        0   
specify      0     0        0    0        0      1       1        1        1   
looking      0     0        0    0        0      0       1        1        1   

         specify  looking  
seems      

In [12]:
# Calculate joint probabilities
total_pairs = sum(term_term_matrix.values())

# Create a new DataFrame for joint probabilities
joint_probabilities_df = pd.DataFrame(index=top_50_words, columns=top_50_words)

for word in top_50_words:
    for context_word in top_50_words:
        pair = (word, context_word)
        count = term_term_matrix.get(pair, 0)
        joint_prob = count / total_pairs
        joint_probabilities_df.at[word, context_word] = joint_prob

# Display the joint probabilities in a table using PrettyTable with formatting

print(joint_probabilities_df)

# Format joint probabilities values to float with 4 decimal places
joint_probabilities_df = joint_probabilities_df.applymap(lambda x: "{:.4f}".format(x) if x != 0 else "0")


            seems      like   request       bit   unclear     could    please  \
seems         0.0  0.014706  0.014706  0.014706  0.014706       0.0       0.0   
like     0.014706       0.0  0.014706  0.014706  0.014706  0.014706       0.0   
request  0.014706  0.014706       0.0  0.014706  0.014706  0.014706  0.014706   
bit      0.014706  0.014706  0.014706       0.0  0.014706  0.014706  0.014706   
unclear  0.014706  0.014706  0.014706  0.014706       0.0  0.014706  0.014706   
could         0.0  0.014706  0.014706  0.014706  0.014706       0.0  0.014706   
please        0.0       0.0  0.014706  0.014706  0.014706  0.014706       0.0   
provide       0.0       0.0       0.0  0.014706  0.014706  0.014706  0.014706   
details       0.0       0.0       0.0       0.0  0.014706  0.014706  0.014706   
specify       0.0       0.0       0.0       0.0       0.0  0.014706  0.014706   
looking       0.0       0.0       0.0       0.0       0.0       0.0  0.014706   

          provide   details

In [13]:
# Calculate marginal probabilities
word_probs = {word: sum(term_term_matrix.get((word, context_word), 0) for context_word in top_50_words) / total_pairs for word in top_50_words}
context_probs = {context_word: sum(term_term_matrix.get((word, context_word), 0) for word in top_50_words) / total_pairs for context_word in top_50_words}

# Create a new DataFrame for PPMI matrix
ppmi_matrix_df = pd.DataFrame(index=top_50_words, columns=top_50_words)

# Calculate PPMI values
for word in top_50_words:
    for context_word in top_50_words:
        pair = (word, context_word)
        joint_prob = term_term_matrix.get(pair, 0) / total_pairs
        word_prob = word_probs[word]
        context_prob = context_probs[context_word]

        if joint_prob > 0 and word_prob > 0 and context_prob > 0:
            ppmi_value = max(math.log2(joint_prob / (word_prob * context_prob)), 0)
            ppmi_matrix_df.at[word, context_word] = ppmi_value
        else:
            ppmi_matrix_df.at[word, context_word] = 0  # Set to 0 if any probability is not positive

# Format PPMI matrix values to float with 4 decimal places
ppmi_matrix_df = ppmi_matrix_df.applymap(lambda x: "{:.4f}".format(x) if x != 0 else "0")
print(ppmi_matrix_df)

          seems    like request     bit unclear   could  please provide  \
seems         0  1.7655  1.5025  1.2801  1.0875       0       0       0   
like     1.7655       0  1.1806  0.9582  0.7655  0.7655       0       0   
request  1.5025  1.1806       0  0.6951  0.5025  0.5025  0.5025       0   
bit      1.2801  0.9582  0.6951       0  0.2801  0.2801  0.2801  0.4728   
unclear  1.0875  0.7655  0.5025  0.2801       0  0.0875  0.0875  0.2801   
could         0  0.7655  0.5025  0.2801  0.0875       0  0.0875  0.2801   
please        0       0  0.5025  0.2801  0.0875  0.0875       0  0.2801   
provide       0       0       0  0.4728  0.2801  0.2801  0.2801       0   
details       0       0       0       0  0.5025  0.5025  0.5025  0.6951   
specify       0       0       0       0       0  0.7655  0.7655  0.9582   
looking       0       0       0       0       0       0  1.0875  1.2801   

        details specify looking  
seems         0       0       0  
like          0       0       0